In [10]:
from gurobipy import *
import numpy as np
import pandas as pd
from HourbidRound1 import *
# from Hourbid import *
from Fibid import *
from blockbid import *

In [11]:
BS_bids = Blockbid()[0]
BD_bids = Blockbid()[1]
HS_bids = HourBid()[0]
HD_bids = HourBid()[1]
FS_bids = Fibid()[0]
FD_bids = Fibid()[1]

/Users/weiliang/Desktop/project/Project-without-E/HourbidRound1.py:47: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  HS_Round=HS_Round.append(Data_H_SP[1000*i:1000*(i+1)].drop_duplicates(subset=['price']))
/Users/weiliang/Desktop/project/Project-without-E/HourbidRound1.py:47: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  HS_Round=HS_Round.append(Data_H_SP[1000*i:1000*(i+1)].drop_duplicates(subset=['price']))
/Users/weiliang/Desktop/project/Project-without-E/HourbidRound1.py:47: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  HS_Round=HS_Round.append(Data_H_SP[1000*i:1000*(i+1)].drop_duplicates(subset=['price']))
/Users/weiliang/Desktop/project/Project-without-E/HourbidRound1.py:47: FutureWarning: The frame.append method is de

In [12]:
HS_bids

,ID,OrderID,Hour,BitType,Quantity,price,Duration,LinkID
0,11,1,1,S,48093.0,0.0,1,NaN
1,11,2,1,S,51455.0,1.0,1,NaN
2,11,3,1,S,53643.0,3.0,1,NaN
3,11,4,1,S,55527.0,5.0,1,NaN
4,11,5,1,S,57628.0,8.0,1,NaN
...,...,...,...,...,...,...,...,...
23037,241,996,24,S,199743.0,1991.0,1,NaN
23038,241,997,24,S,199777.0,1993.0,1,NaN
23039,241,998,24,S,199846.0,1995.0,1,NaN
23040,241,999,24,S,199931.0,1998.0,1,NaN


In [13]:
find_largest(HIndex_D)

972

In [14]:

# Import the bids data


# length of the bids
L_D = len(BD_bids)
L_S = len(Blockbid()[0])

NoneChild_D=[]
Child_D = []
Parent_D=[]
NoneChild_S=[]
Child_S=[]
Parent_S=[]
for i in range(L_D):
    if BD_bids.isnull()['LinkID'][i] == True:
        NoneChild_D.append(i)
    else:
        Parent_D.append(np.where(BD_bids['ID']==BD_bids['LinkID'][i])[0][0])
        Child_D.append(i)
for i in range(L_S):
    if BS_bids.isnull()['LinkID'][i] == True:
        NoneChild_S.append(i)
    else:
        Parent_S.append(np.where(BS_bids['ID']==BS_bids['LinkID'][i])[0][0])
        Child_S.append(i)
        

L_FD = len(Fibid()[1])
L_FS = len(Fibid()[0])
J = np.arange(1,find_largest(HIndex_D)+1)
K = np.arange(1,find_largest(HIndex_S)+1)

# 24 Hours 
H = np.arange(1,25)


#Delta variables
Delta_S = np.zeros([L_S,24])
Delta_D = np.zeros([L_D,24])
# For Demand
for i in range (L_D):
    if BD_bids['Hour'][i]+BD_bids['Duration'][i]<=25:
        Delta_D[i,BD_bids['Hour'][i]-1:(BD_bids['Hour'][i]+BD_bids['Duration'][i])-1] = 1
#For Demand
for i in range (L_S):
    if BS_bids['Hour'][i]+BS_bids['Duration'][i]<=25:
        Delta_S[i,BS_bids['Hour'][i]-1:(BS_bids['Hour'][i]+BS_bids['Duration'][i])-1] = 1
        
#Find the Max and min value for each hour        
def find_F(Del,leng,bids):
    Res = [[] for i in range(24)]
    Res_max = [[] for i in range(24)]
    Res_min = [[] for i in range(24)]
    for i in range (24):
        for j in range(leng):
            if Del[j,i] == 1:
                Res[i].append(bids['price'][j])
    for i in range(24):
        Res_max[i] = np.max(Res[i])
        Res_min[i] = np.min(Res[i])
    return(Res_max,Res_min)
# Max and min for flexible bids
FD_min = np.min(FD_bids['price'])
FD_max = np.max(FD_bids['price'])
FS_min = np.min(FS_bids['price'])
FS_max = np.max(FS_bids['price'])

# Find the max and min for block bid and stored in the form like : [Hour,Block,Flex]
DMax_Res = [[] for i in range(24)]
Dm_Res = [[] for i in range(24)]
SMax_Res = [[] for i in range(24)]
Sm_Res = [[] for i in range(24)]

for i in range(24):
    DMax_Res[i].append(HD_bids['price'][HIndex_D[i][0]])
    DMax_Res[i].append(find_F(Delta_D,L_D,BD_bids)[0][i])
    DMax_Res[i].append(FD_max)
    
for i in range(24):
    SMax_Res[i].append(HS_bids['price'][HIndex_S[i][1]])
    SMax_Res[i].append(find_F(Delta_S,L_S,BS_bids)[0][i])
    SMax_Res[i].append(FS_max)
    
for i in range(24):
    Dm_Res[i].append(HD_bids['price'][HIndex_D[i][1]])
    Dm_Res[i].append(find_F(Delta_D,L_D,BD_bids)[1][i])
    Dm_Res[i].append(FD_min)
    
for i in range(24):
    Sm_Res[i].append(HS_bids['price'][HIndex_S[i][0]])
    Sm_Res[i].append(find_F(Delta_S,L_S,BS_bids)[1][i])
    Sm_Res[i].append(FS_min)
    
# Create the Model    
E = Model("Energy")
E.params.NonConvex =2
E.update()

# Add variables
# solution variables X for demand and supply in Hourly Bids
xD = E.addVars(H,J,ub=1,lb=0,  vtype=GRB.CONTINUOUS, name='xD')
xS = E.addVars(H,K,ub=1,lb=0,  vtype=GRB.CONTINUOUS, name='xS')

# Auxiliary variables for demand and supply
w_D = E.addVars(H,J,ub=1,lb=0,  vtype=GRB.BINARY,name ='W_D')
w_S = E.addVars(H,K,ub=1,lb=0,  vtype=GRB.BINARY,name ='W_S')

#solution variables Y for demand and supply in Block Bids
yD = E.addVars(L_D,ub=1,lb=0,  vtype=GRB.BINARY,name ='yD')
yS = E.addVars(L_S,ub=1,lb=0,  vtype=GRB.BINARY,name ='yS')

# Market clearing price at time H for demand and supply
F_z = E.addVars(H,lb = 0,vtype=GRB.CONTINUOUS, name='F_ZS')

# solution variables V for demand and supply in Hourly Bids
V_S = E.addVars(H,L_FS,lb=0,vtype=GRB.BINARY,name = 'V_S')
V_D = E.addVars(H,L_FD,lb=0,vtype=GRB.BINARY,name = 'V_D')

# Add Objective Function
# Hour Demand Part 
# Hour Supply Part
# Block Part
# Flexbile Part
# First segment for Hour Demand
E.setObjective(sum(0.5*(2*Pair(j,h,HD_bids,'DP')[0]+xD[h,j]*(Pair(j,h,HD_bids,'DP')[1]-Pair(j,h,HD_bids,'DP')[0]))*xD[h,j]*(Pair(j,h,HD_bids,'DQ')[1]-Pair(j,h,HD_bids,'DQ')[0])for j in J for h in H)
                -sum(0.5*(2*Pair(k,h,HS_bids,'SP')[0]+xS[h,k]*(Pair(k,h,HS_bids,'SP')[1]-Pair(k,h,HS_bids,'SP')[0]))*xS[h,k]*(Pair(k,h,HS_bids,'SQ')[1]-Pair(k,h,HS_bids,'SQ')[0])for k in K for h in H)
                +sum(BD_bids['Duration'][i]*BD_bids['Quantity'][i]*BD_bids['price'][i]*yD[i] for i in range(L_D)) - sum(BS_bids['Duration'][i]*BS_bids['Quantity'][i]*BS_bids['price'][i]*yS[i] for i in range(L_S))
                +sum(FD_bids['Quantity'][i]*FD_bids['price'][i] for i in range(L_FD))*sum(V_D[h,i]for i in range(L_FD) for h in H)-sum(FS_bids['Quantity'][i]*FS_bids['price'][i] for i in range(L_FS))*sum(V_S[h,i]for i in range(L_FS) for h in H)
                +sum(Pair(1,h,HD_bids,'DQ')[0]-Pair(1,h,HD_bids,'DP')[0] for h in H)
                
,GRB.MAXIMIZE)

gamma = 1000000

        # (2)
E.addConstrs(-sum((Pair(j,h,HD_bids,'DQ')[1]-Pair(j,h,HD_bids,'DQ')[0]) * xD[h,j] for j in J ) +sum((Pair(k,h,HS_bids,'SQ')[1]-Pair(k,h,HS_bids,'SQ')[0])*xS[h,k] for k in K) -Pair(1,h,HD_bids,'DQ')[0] +Pair(1,h,HS_bids,'SQ')[0] 
                    +sum(Delta_S[i,h-1]*BS_bids['Quantity'][i]*yS[i] for i in range(L_S))  -sum(Delta_D[i,h-1]*BD_bids['Quantity'][i]*yD[i] for i in range(L_D))
                    +sum(FS_bids['Quantity'][i]*V_S[h,i]for i in range(L_FS))-sum(FD_bids['Quantity'][i]*V_D[h,i]for i in range(L_FD))
                    
                    ==0 for h in H) 
 


E.addConstrs(F_z[h] == Sm_Res[h-1][0]+sum((Pair(k,h,HS_bids,'SP')[1]-Pair(k,h,HS_bids,'SP')[0]) * xS[h,k] for k in K)for h in H)        
E.addConstrs(F_z[h] == DMax_Res[h-1][0]+sum((Pair(j,h,HD_bids,'DP')[1]-Pair(j,h,HD_bids,'DP')[0]) * xD[h,j] for j in J) for h in H)
# (8)

E.addConstrs(-BS_bids['Duration'][j]* BS_bids['price'][j]+ sum(Delta_S[j,h-1]*F_z[h] for h in H ) <= yS[j]*gamma for j in NoneChild_S)
# (9)

E.addConstrs(BD_bids['Duration'][j]*BD_bids['price'][j]- sum(Delta_S[j,h-1]*F_z[h] for h in H) <= yD[j]*gamma for j in NoneChild_D)
        
    # E.addConstr(if Blockbid['LinkID'][j] == 'NaN': Blockbid['Duration'][j]*Blockbid['price'][j]+ sum(Delta_S[j,h]*F_z[h] for j in J) <= yd[j]*gamma)
        
# (3)(4)
# Demand
E.addConstrs(w_D[h,1]<=xD[h,1] for h in H)
E.addConstrs(xD[h,1] <=1 for h in H)
E.addConstrs(w_D[h,j]<=xD[h,j] for j in range(2,find_largest(HIndex_D)) for h in H)
E.addConstrs(xD[h,j]<=w_D[h,j-1] for j in range(2,find_largest(HIndex_D)) for h in H)
E.addConstrs(0<= xD[h,J[-1]] for h in H)
E.addConstrs(xD[h,J[-1]] <= w_D[h,J[-1]-1] for h in H)

# Supply
E.addConstrs(w_S[h,1]<=xS[h,1] for h in H)
E.addConstrs(xS[h,1] <=1 for h in H)
E.addConstrs(w_S[h,k]<=xS[h,k] for k in range(2,find_largest(HIndex_S)) for h in H)
E.addConstrs(xS[h,k]<=w_S[h,k-1] for k in range(2,find_largest(HIndex_S)) for h in H)
E.addConstrs(0<= xS[h,K[-1]] for h in H)
E.addConstrs(xS[h,K[-1]] <= w_S[h,K[-1]-1] for h in H)


E.addConstrs(sum(V_S[h,i] for h in H )<= 1 for i in range(L_FS) )
E.addConstrs(sum(V_D[h,i] for h in H )<= 1 for i in range(L_FD) )

E.addConstrs(F_z[h]-FS_bids['price'][i] <= gamma*sum(V_S[k,i] for k in H) for i in range(L_FS) for h in H)
E.addConstrs(FD_bids['price'][i]-F_z[h]<=gamma*sum(V_D[k,i] for k in H) for i in range(L_FD) for h in H)

E.addConstrs(yD[Child_D[i]]<=yD[Parent_D[i]] for i in range(len(Child_D)))
E.addConstrs(yS[Child_S[i]]<=yS[Parent_S[i]] for i in range(len(Child_S)))





Set parameter NonConvex to value 2


{0: <gurobi.Constr *Awaiting Model Update*>,
 1: <gurobi.Constr *Awaiting Model Update*>,
 2: <gurobi.Constr *Awaiting Model Update*>,
 3: <gurobi.Constr *Awaiting Model Update*>,
 4: <gurobi.Constr *Awaiting Model Update*>,
 5: <gurobi.Constr *Awaiting Model Update*>,
 6: <gurobi.Constr *Awaiting Model Update*>,
 7: <gurobi.Constr *Awaiting Model Update*>,
 8: <gurobi.Constr *Awaiting Model Update*>,
 9: <gurobi.Constr *Awaiting Model Update*>,
 10: <gurobi.Constr *Awaiting Model Update*>,
 11: <gurobi.Constr *Awaiting Model Update*>,
 12: <gurobi.Constr *Awaiting Model Update*>,
 13: <gurobi.Constr *Awaiting Model Update*>,
 14: <gurobi.Constr *Awaiting Model Update*>,
 15: <gurobi.Constr *Awaiting Model Update*>,
 16: <gurobi.Constr *Awaiting Model Update*>,
 17: <gurobi.Constr *Awaiting Model Update*>,
 18: <gurobi.Constr *Awaiting Model Update*>,
 19: <gurobi.Constr *Awaiting Model Update*>,
 20: <gurobi.Constr *Awaiting Model Update*>,
 21: <gurobi.Constr *Awaiting Model Update*>

In [15]:
HS_bids

,ID,OrderID,Hour,BitType,Quantity,price,Duration,LinkID
0,11,1,1,S,48093.0,0.0,1,NaN
1,11,2,1,S,51455.0,1.0,1,NaN
2,11,3,1,S,53643.0,3.0,1,NaN
3,11,4,1,S,55527.0,5.0,1,NaN
4,11,5,1,S,57628.0,8.0,1,NaN
...,...,...,...,...,...,...,...,...
23037,241,996,24,S,199743.0,1991.0,1,NaN
23038,241,997,24,S,199777.0,1993.0,1,NaN
23039,241,998,24,S,199846.0,1995.0,1,NaN
23040,241,999,24,S,199931.0,1998.0,1,NaN


In [16]:

E.optimize()

Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (mac64[arm])
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 94738 rows, 94640 columns and 315326 nonzeros
Model fingerprint: 0x24a0dfa2
Model has 45971 quadratic objective terms
Variable types: 46632 continuous, 48008 integer (48008 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+06]
  Objective range  [6e+02, 2e+07]
  QObjective range [1e+00, 2e+04]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+05]
Presolve removed 10680 rows and 10108 columns
Presolve time: 1.53s
Presolved: 84058 rows, 84532 columns, 257589 nonzeros
Presolved model has 41565 quadratic objective terms
Variable types: 41589 continuous, 42943 integer (42943 binary)

Root simplex log...

Iteration    Objective       Primal Inf.    Dual Inf.      Time
   21856    1.2949095e+09   0.000000e+00   1.570468e+09      5s
   50012    2.1891952e+09   0.000000e+00   3.659663e+07     10s
   94760    3.743

In [26]:
E.getAttr('x',yS)

{0: 1.0,
 1: -0.0,
 2: 1.0,
 3: 1.0,
 4: -0.0,
 5: 1.0,
 6: -0.0,
 7: 1.0,
 8: 1.0,
 9: 1.0,
 10: -0.0,
 11: 0.0,
 12: 1.0,
 13: 1.0,
 14: -0.0,
 15: -0.0,
 16: 1.0,
 17: 1.0,
 18: -0.0,
 19: -0.0,
 20: 1.0,
 21: -0.0,
 22: 1.0,
 23: -0.0,
 24: 1.0,
 25: -0.0,
 26: 1.0,
 27: 1.0,
 28: 1.0,
 29: 1.0,
 30: 1.0,
 31: 1.0,
 32: 1.0,
 33: 1.0,
 34: 1.0,
 35: 1.0,
 36: -0.0,
 37: -0.0,
 38: -0.0,
 39: -0.0,
 40: 1.0,
 41: -0.0,
 42: 1.0,
 43: 1.0,
 44: 1.0,
 45: 1.0,
 46: 1.0,
 47: -0.0,
 48: -0.0,
 49: 1.0,
 50: -0.0,
 51: 1.0,
 52: 1.0,
 53: 1.0,
 54: -0.0,
 55: 1.0,
 56: 1.0,
 57: 1.0,
 58: 1.0,
 59: 0.0,
 60: -0.0,
 61: 1.0,
 62: 1.0,
 63: 1.0,
 64: 1.0,
 65: -0.0,
 66: 1.0,
 67: 1.0,
 68: 1.0,
 69: -0.0,
 70: 1.0,
 71: -0.0,
 72: 1.0,
 73: -0.0,
 74: -0.0,
 75: 1.0,
 76: 1.0,
 77: 1.0,
 78: 1.0,
 79: 1.0,
 80: -0.0,
 81: 1.0,
 82: 1.0,
 83: 1.0,
 84: -0.0,
 85: 1.0,
 86: 1.0,
 87: 1.0,
 88: 1.0,
 89: 1.0,
 90: 1.0,
 91: 1.0,
 92: 1.0,
 93: 1.0,
 94: 1.0,
 95: 1.0,
 96: 1.0,
 97: 1.0,
 9

In [22]:
len(FD_bids)

19